Instacart

This workbook about feature generation
The generated fetaures are stored as csv files, so can be loaded from subseqent pages

**TODO**
Combine the feature generation for train and test.


## 1.  Load data

In [3]:
import numpy as np 
import pandas as pd 
import time
from tqdm import tqdm

import gc
IDIR = './data/'


print('loading prior')
priors = pd.read_csv(IDIR + 'order_products__prior.csv')
print('loading train')
train = pd.read_csv(IDIR + 'order_products__train.csv')
print('loading orders')
orders = pd.read_csv(IDIR + 'orders.csv')

print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))

###
# some memory measures for kaggle kernel
print('optimize memory')
orders.order_dow = orders.order_dow.astype(np.int8)
orders.order_hour_of_day = orders.order_hour_of_day.astype(np.int8)
orders.order_number = orders.order_number.astype(np.int16)
orders.order_id = orders.order_id.astype(np.int32)
orders.user_id = orders.user_id.astype(np.int32)
orders.days_since_prior_order = orders.days_since_prior_order.astype(np.float32)


train.reordered = train.reordered.astype(np.int8)
train.add_to_cart_order = train.add_to_cart_order.astype(np.int16)

priors.order_id = priors.order_id.astype(np.int32)
priors.add_to_cart_order = priors.add_to_cart_order.astype(np.int16)
priors.reordered = priors.reordered.astype(np.int8)
priors.product_id = priors.product_id.astype(np.int32)


loading prior
loading train
loading orders
priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered
optimize memory


### Load Products

In [4]:
print('loading products')


products = pd.read_csv(IDIR + 'products.csv')

products.drop(['product_name'], axis=1, inplace=True)
products.aisle_id = products.aisle_id.astype(np.int8)
products.department_id = products.department_id.astype(np.int8)
products.product_id = products.product_id.astype(np.int32)


loading products


### Concat prior and train 

To be used for final predictions

In [5]:
prior_train = pd.concat([priors, train], axis = 0)
print prior_train.shape

if ( priors.shape[0] + train.shape[0] ) ==  prior_train.shape[0]:
    print "concat successful"


(33819106, 4)
concat successful


------------------------

## 2. Prepare data

### Join with orders table to get the user id

This gets us the user_id and other order features

In [6]:
## set the index, for the join
orders.set_index('order_id', inplace=True, drop=False)

## Join with prior_train
print "Joining orders with prior_train"
prior_train = prior_train.join(orders, on = 'order_id', how = 'left', lsuffix = '_')
prior_train.drop('order_id_', inplace = True, axis = 1)
prior_train.head()



Joining orders with prior_train


,product_id,add_to_cart_order,reordered,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,33120,1,1,2,202279,prior,3,5,9,8.0
1,28985,2,1,2,202279,prior,3,5,9,8.0
2,9327,3,0,2,202279,prior,3,5,9,8.0
3,45918,4,1,2,202279,prior,3,5,9,8.0
4,30035,5,0,2,202279,prior,3,5,9,8.0


In [7]:
## Repeat the same only for prior

print "Joining orders with priors"
priors = priors.join(orders, on = 'order_id', how = 'left', lsuffix = '_')
priors.drop('order_id_', inplace = True, axis = 1)
priors.head()


Joining orders with priors


,product_id,add_to_cart_order,reordered,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,33120,1,1,2,202279,prior,3,5,9,8.0
1,28985,2,1,2,202279,prior,3,5,9,8.0
2,9327,3,0,2,202279,prior,3,5,9,8.0
3,45918,4,1,2,202279,prior,3,5,9,8.0
4,30035,5,0,2,202279,prior,3,5,9,8.0


### Make a data frame of user and previous product list

In [8]:

users_prior_all = pd.DataFrame()
users_prior_all['prod_list'] = prior_train.groupby('user_id')['product_id'].apply(set)
users_prior_all.reset_index(inplace = True, drop = False)
users_prior_all.head()

,user_id,prod_list
0,1,"{17122, 196, 26405, 27845, 13032, 39657, 12427..."
1,2,"{45066, 2573, 18961, 23, 32792, 22559, 12324, ..."
2,3,"{17668, 39190, 44683, 21903, 14992, 21137, 324..."
3,4,"{26576, 21573, 17769, 25623, 35469, 37646, 366..."
4,5,"{11777, 40706, 48775, 20754, 28289, 6808, 1398..."


In [9]:
users_prior = pd.DataFrame()
users_prior['prod_list'] = priors.groupby('user_id')['product_id'].apply(set)
users_prior.reset_index(inplace = True, drop = False)
users_prior.head()

,user_id,prod_list
0,1,"{17122, 196, 26405, 13032, 39657, 12427, 25133..."
1,2,"{45066, 2573, 18961, 23, 32792, 22559, 13351, ..."
2,3,"{17668, 39190, 44683, 21903, 14992, 21137, 324..."
3,4,"{26576, 21573, 17769, 25623, 35469, 37646, 366..."
4,5,"{11777, 40706, 48775, 20754, 28289, 6808, 1398..."


In [10]:
## Prepare the data in the same way for train

print "user train product list"
train = train.join(orders, on = 'order_id', how = "left", lsuffix ="_")
train.drop('order_id_', inplace=True, axis=1)


users_train = pd.DataFrame()
users_train['prod_list'] = train.groupby('user_id')['product_id'].apply(set)
users_train.reset_index(inplace = True, drop = False)

users_train.head()

user train product list


,user_id,prod_list
0,1,"{196, 26405, 13032, 39657, 25133, 38928, 26088..."
1,2,"{24838, 11913, 45066, 31883, 38547, 24852, 327..."
2,5,"{40706, 21413, 20843, 48204, 21616, 19057, 201..."
3,7,"{29894, 17638, 47272, 45066, 13198, 37999, 408..."
4,8,"{27104, 15937, 5539, 41540, 31717, 48230, 2224..."


In [11]:
## reset the order table index
orders.reset_index(inplace=True, drop=True)
orders.head()



,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


-----------------------------------

## 3. Prepare Y Variable


* Here we use the train table as our y variable. We create a dataset with user_id, product_id, is_product_in_train columns.

* We can then use this to merge with user, product and user/product features.

* This will serve as our testing data before use actually run our model on real data.

In [12]:
## This block needs to be run only once
## Later the output of this block is stored in ./data/features.csv file
## The next block reads the file, it enough to run the next block subsequently

"""


train_list = orders[orders.eval_set == 'train']

print "features"
feature = []
count = 0
for user_record in (train_list.itertuples()):
	count+=1
	if count%10000 == 0:
		print "Finished {} users".format(count)
	
	user_id = user_record.user_id
	order_id = user_record.order_id 
	products      = list(users_train[users_train.user_id == user_id]['prod_list'].values.tolist()[0])
	prev_products = list(users_prior[users_prior.user_id == user_id]['prod_list'].values.tolist()[0])

	for p_p in prev_products:
		if p_p in products:
			feature.append((user_id, p_p, order_id ,1))
		else:
			feature.append((user_id, p_p, order_id, 0))

feature_df = pd.DataFrame(data = feature, columns =['user_id','product_id','nxt_order_id','in_next_order'])

print feature_df.head()

feature_df.to_csv('./data/features.csv')

"""

'\n\n\ntrain_list = orders[orders.eval_set == \'train\']\n\nprint "features"\nfeature = []\ncount = 0\nfor user_record in (train_list.itertuples()):\n\tcount+=1\n\tif count%10000 == 0:\n\t\tprint "Finished {} users".format(count)\n\t\n\tuser_id = user_record.user_id\n\torder_id = user_record.order_id \n\tproducts      = list(users_train[users_train.user_id == user_id][\'prod_list\'].values.tolist()[0])\n\tprev_products = list(users_prior[users_prior.user_id == user_id][\'prod_list\'].values.tolist()[0])\n\n\tfor p_p in prev_products:\n\t\tif p_p in products:\n\t\t\tfeature.append((user_id, p_p, order_id ,1))\n\t\telse:\n\t\t\tfeature.append((user_id, p_p, order_id, 0))\n\nfeature_df = pd.DataFrame(data = feature, columns =[\'user_id\',\'product_id\',\'nxt_order_id\',\'in_next_order\'])\n\nprint feature_df.head()\n\nfeature_df.to_csv(\'./data/features.csv\')\n\n'

In [13]:
features = pd.read_csv('./data/features.csv')
features.drop(features.columns[0], inplace=True, axis=1)
features.head()

,user_id,product_id,nxt_order_id,in_next_order
0,1,17122,1187899,0
1,1,196,1187899,1
2,1,26405,1187899,1
3,1,13032,1187899,1
4,1,39657,1187899,1


-----------------------------

## 4.  Feature Generation

###  Order Features

In [14]:
print "Order features"

features.set_index('nxt_order_id',inplace = True, drop = False)
orders.set_index('order_id', inplace = True, drop = False)

features = pd.merge(features, orders, left_on = 'nxt_order_id', right_on = 'order_id')

features.drop('order_id', inplace = True, axis =1)
features.drop('eval_set', inplace = True, axis =1)
features.drop('user_id_y', inplace = True, axis =1)
features.drop('order_number', inplace = True, axis =1)



features = features.rename(columns={"user_id_x":"user_id"})


features.reset_index(drop = True, inplace= True)
train.reset_index(drop = True, inplace = True)

features.head()

Order features


,user_id,product_id,nxt_order_id,in_next_order,order_dow,order_hour_of_day,days_since_prior_order
0,1,17122,1187899,0,4,8,14.0
1,1,196,1187899,1,4,8,14.0
2,1,26405,1187899,1,4,8,14.0
3,1,13032,1187899,1,4,8,14.0
4,1,39657,1187899,1,4,8,14.0


### User id Order id features

In [15]:
## using prior stats

user_order_prior = pd.DataFrame()
user_order_prior['avg_days_since_prior_order'] = priors.groupby('user_id')['days_since_prior_order'].agg('mean')

## using prior train stats
user_order_all = pd.DataFrame()
user_order_all['avg_days_since_prior_order'] = prior_train.groupby('user_id')['days_since_prior_order'].agg('mean')

user_order_prior.reset_index(drop = False, inplace = True)
user_order_all.reset_index(drop = False, inplace = True)

print user_order_prior.head()
print user_order_all.head()


   user_id  avg_days_since_prior_order
0        1                   20.259260
1        2                   15.967033
2        3                   11.487180
3        4                   15.357142
4        5                   14.500000
   user_id  avg_days_since_prior_order
0        1                   19.200001
1        2                   18.009390
2        3                   11.487180
3        4                   15.357142
4        5                   12.314285


### User id last order feature

In [17]:
## Stats from prior

max_order = pd.DataFrame()
max_order['max_order'] = priors.groupby(['user_id'])['order_number'].agg('max')
max_order = max_order.rename(columns = {"max_order":"order_number"})
max_order.reset_index(inplace = True, drop = False)


#max_order.head()


max_order.set_index(['user_id','order_number'], drop = False, inplace = True)
priors.set_index(['user_id','order_number'], drop = False, inplace = True)

max_order = max_order.join(priors[['user_id', 'order_id', 'order_number','order_dow','order_hour_of_day','days_since_prior_order']], rsuffix="_")

max_order.reset_index(drop =True, inplace = True)
priors.reset_index(drop = True, inplace = True)

max_order.drop('user_id_', inplace = True, axis =1)
max_order.drop('order_number_', inplace = True, axis = 1)
max_order.drop('order_number', inplace = True, axis = 1)


max_order = max_order.rename(columns = {"order_id":"prev_order_id","order_dow":"prev_order_dow"
                                       ,"order_hour_of_day":"prev_order_hour_of_day"
                                       ,"days_since_prior_order":"prev_days_since_prior_order"})

max_order.drop_duplicates(inplace = True)
max_order.head()



,user_id,prev_order_id,prev_order_dow,prev_order_hour_of_day,prev_days_since_prior_order
0,1,2550362,4,8,30.0
9,2,839880,3,10,13.0
25,3,1402502,1,15,15.0
31,4,2557754,5,13,0.0
34,5,157374,1,18,19.0


In [18]:
## Stats from prior and train

max_order_all = pd.DataFrame()
max_order_all['max_order'] = prior_train.groupby(['user_id'])['order_number'].agg('max')
max_order_all = max_order_all.rename(columns = {"max_order":"order_number"})
max_order_all.reset_index(inplace = True, drop = False)


#max_order.head()


max_order_all.set_index(['user_id','order_number'], drop = False, inplace = True)
prior_train.set_index(['user_id','order_number'], drop = False, inplace = True)

max_order_all = max_order_all.join(prior_train[['user_id', 'order_id', 'order_number','order_dow','order_hour_of_day','days_since_prior_order']], rsuffix="_")

max_order_all.reset_index(drop =True, inplace = True)
prior_train.reset_index(drop = True, inplace = True)

max_order_all.drop('user_id_', inplace = True, axis =1)
max_order_all.drop('order_number_', inplace = True, axis = 1)
max_order_all.drop('order_number', inplace = True, axis = 1)


max_order_all = max_order_all.rename(columns = {"order_id":"prev_order_id","order_dow":"prev_order_dow"
                                       ,"order_hour_of_day":"prev_order_hour_of_day"
                                       ,"days_since_prior_order":"prev_days_since_prior_order"})

max_order_all.drop_duplicates(inplace = True)
max_order_all.head()

,user_id,prev_order_id,prev_order_dow,prev_order_hour_of_day,prev_days_since_prior_order
0,1,1187899,4,8,14.0
11,2,1492625,1,11,30.0
42,3,1402502,1,15,15.0
48,4,2557754,5,13,0.0
51,5,2196797,0,11,6.0


###  Product Features

In [19]:
print "Product Features"

### Stats from prior and train
prods = pd.DataFrame()
prods['p_orders'] = prior_train.groupby(prior_train.product_id).size().astype(np.float32)
prods['p_reorders'] = prior_train['reordered'].groupby(prior_train.product_id).sum().astype(np.float32)

prods['p_reorder_rate'] = (prods.p_reorders / prods.p_orders).astype(np.float32)

## set the index for products
products.set_index('product_id', inplace = True, drop = False)
products_all = products.join(prods, rsuffix="_")
## Reset the index
products_all.reset_index(inplace = True, drop = True)
del prods
products_all.head()







Product Features


,product_id,aisle_id,department_id,p_orders,p_reorders,p_reorder_rate
0,1,61,19,1928.0,1185.0,0.614627
1,2,104,13,94.0,13.0,0.138298
2,3,94,7,283.0,209.0,0.738516
3,4,38,1,351.0,161.0,0.458689
4,5,5,13,16.0,10.0,0.625000


In [20]:
## Stats from only prior

prods = pd.DataFrame()
prods['p_orders'] = priors.groupby(priors.product_id).size().astype(np.float32)
prods['p_reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)

prods['p_reorder_rate'] = (prods.p_reorders / prods.p_orders).astype(np.float32)

## set the index for products
products.set_index('product_id', inplace = True, drop = False)
products_prior = products.join(prods, rsuffix="_")
## Reset the index
products_prior.reset_index(inplace = True, drop = True)
products_prior.head()





,product_id,aisle_id,department_id,p_orders,p_reorders,p_reorder_rate
0,1,61,19,1852.0,1136.0,0.613391
1,2,104,13,90.0,12.0,0.133333
2,3,94,7,277.0,203.0,0.732852
3,4,38,1,329.0,147.0,0.446809
4,5,5,13,15.0,9.0,0.600000


### User features

In [21]:
## Stats from only prior

prod_count_prior = pd.DataFrame()

prod_count_prior['basket_size'] = priors.groupby(['user_id','order_id'])['product_id'].size().astype(np.int32)
prod_count_prior['reorder_size'] = priors.groupby(['user_id','order_id'])['reordered'].agg('sum').astype(np.int32)

# reset / set index
prod_count_prior = prod_count_prior.reset_index()
prod_count_prior.set_index('user_id', inplace = True, drop =False)

prod_count_prior['tot_orders']  =  prod_count_prior.groupby(['user_id']).size().astype(np.int32)
prod_count_prior['tot_prods']   =  prod_count_prior.groupby(['user_id'])['basket_size'].agg(['sum'])
prod_count_prior['avg_basket']  =  prod_count_prior.groupby(['user_id'])['basket_size'].agg(['mean'])
prod_count_prior['avg_reorder'] =  prod_count_prior.groupby(['user_id'])['reorder_size'].agg(['mean'])
prod_count_prior['std_basket']  =  prod_count_prior.groupby(['user_id'])['basket_size'].agg(['std'])


prod_count_prior.drop(['order_id','basket_size','reorder_size'], inplace=True, axis=1)
prod_count_prior.drop_duplicates(inplace = True)
prod_count_prior = prod_count_prior.reset_index(level = 'user_id', drop = True)

prod_count_prior.head()


,user_id,tot_orders,tot_prods,avg_basket,avg_reorder,std_basket
0,1,10,59,5.900000,4.100000,1.523884
1,2,14,195,13.928571,6.642857,5.717238
2,3,12,88,7.333333,4.583333,2.103388
3,4,5,18,3.600000,0.200000,2.073644
4,5,4,37,9.250000,3.500000,3.095696


In [22]:
## Stats from all



prod_count_all = pd.DataFrame()

prod_count_all['basket_size']  = prior_train.groupby(['user_id','order_id'])['product_id'].size().astype(np.int32)
prod_count_all['reorder_size'] = prior_train.groupby(['user_id','order_id'])['reordered'].agg('sum').astype(np.int32)

# reset / set index
prod_count_all = prod_count_all.reset_index()
prod_count_all.set_index('user_id', inplace = True, drop =False)

prod_count_all['tot_orders']  =  prod_count_all.groupby(['user_id']).size().astype(np.int32)
prod_count_all['tot_prods']   =  prod_count_all.groupby(['user_id'])['basket_size'].agg(['sum'])
prod_count_all['avg_basket']  =  prod_count_all.groupby(['user_id'])['basket_size'].agg(['mean'])
prod_count_all['avg_reorder'] =  prod_count_all.groupby(['user_id'])['reorder_size'].agg(['mean'])
prod_count_all['std_basket']  =  prod_count_all.groupby(['user_id'])['basket_size'].agg(['std'])


prod_count_all.drop(['order_id','basket_size','reorder_size'], inplace=True, axis=1)
prod_count_all.drop_duplicates(inplace = True)
prod_count_all = prod_count_all.reset_index(level = 'user_id', drop = True)

prod_count_all.head()


,user_id,tot_orders,tot_prods,avg_basket,avg_reorder,std_basket
0,1,11,70,6.363636,4.636364,2.110579
1,2,15,226,15.066667,7.000000,7.055562
2,3,12,88,7.333333,4.583333,2.103388
3,4,5,18,3.600000,0.200000,2.073644
4,5,5,46,9.200000,3.600000,2.683282


### User Product Features

In [23]:
## Stats from prior

user_prod_prior = pd.DataFrame()
user_prod_prior['up_orders'] = priors.groupby(['user_id','product_id'])['order_id'].size()

user_prod_prior.reset_index(inplace = True, drop = False)
user_prod_prior.set_index(['user_id', 'product_id'], inplace = True, drop = False)

user_prod_prior['up_reorder']      = priors.groupby(['user_id', 'product_id'])['reordered'].agg(['sum'])
user_prod_prior['up_reorder_rate'] = user_prod_prior.up_reorder / user_prod_prior.up_orders

user_prod_prior.reset_index(drop = True, inplace= True)

user_prod_prior.head()

,user_id,product_id,up_orders,up_reorder,up_reorder_rate
0,1,196,10,9,0.900000
1,1,10258,9,8,0.888889
2,1,10326,1,0,0.000000
3,1,12427,10,9,0.900000
4,1,13032,3,2,0.666667


In [24]:
## Stats from all

user_prod_all = pd.DataFrame()
user_prod_all['up_orders'] = prior_train.groupby(['user_id','product_id'])['order_id'].size()

user_prod_all.reset_index(inplace = True, drop = False)
user_prod_all.set_index(['user_id', 'product_id'], inplace = True, drop = False)

user_prod_all['up_reorder']      = prior_train.groupby(['user_id', 'product_id'])['reordered'].agg(['sum'])
user_prod_all['up_reorder_rate'] = user_prod_all.up_reorder / user_prod_all.up_orders

user_prod_all.reset_index(drop = True, inplace= True)

user_prod_all.head()

,user_id,product_id,up_orders,up_reorder,up_reorder_rate
0,1,196,11,10,0.909091
1,1,10258,10,9,0.900000
2,1,10326,1,0,0.000000
3,1,12427,10,9,0.900000
4,1,13032,4,3,0.750000


## 5. Join the features

### Join User Id Order features

In [26]:
features.set_index('user_id', drop = False, inplace = True)
user_order_prior.set_index('user_id', drop = False, inplace = True)

features = features.join(user_order_prior,on ='user_id', rsuffix='_')
features.drop('user_id_', inplace = True, axis = 1)

features.reset_index(drop = True, inplace = True)
user_order_prior.reset_index(drop = True, inplace = True)

features.head()

,user_id,product_id,nxt_order_id,in_next_order,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order
0,1,17122,1187899,0,4,8,14.0,20.25926
1,1,196,1187899,1,4,8,14.0,20.25926
2,1,26405,1187899,1,4,8,14.0,20.25926
3,1,13032,1187899,1,4,8,14.0,20.25926
4,1,39657,1187899,1,4,8,14.0,20.25926


### User id last order id feature

In [28]:
features.set_index('user_id', drop = False, inplace = True)
max_order.set_index('user_id', drop = False, inplace = True)

features = features.join(max_order, on ='user_id', rsuffix='_')
features.drop("user_id_", inplace = True, axis = 1)
features.drop("prev_order_id", inplace = True, axis = 1)

features.reset_index(drop = True, inplace = True)
max_order.reset_index(drop = True, inplace = True)

features.head()


,user_id,product_id,nxt_order_id,in_next_order,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,prev_days_since_prior_order
0,1,17122,1187899,0,4,8,14.0,20.25926,4,8,30.0
1,1,196,1187899,1,4,8,14.0,20.25926,4,8,30.0
2,1,26405,1187899,1,4,8,14.0,20.25926,4,8,30.0
3,1,13032,1187899,1,4,8,14.0,20.25926,4,8,30.0
4,1,39657,1187899,1,4,8,14.0,20.25926,4,8,30.0


### Join product features

In [29]:
features.set_index('product_id', drop = False, inplace=True)
products_prior.set_index('product_id', drop = False, inplace=True)


features = features.join(products_prior, rsuffix='_')
features.drop('product_id_', inplace=True, axis = 1)

features.reset_index(inplace = True, drop = True)
products_prior.reset_index(inplace = True, drop = True)



features.head()

,user_id,product_id,nxt_order_id,in_next_order,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,prev_days_since_prior_order,aisle_id,department_id,p_orders,p_reorders,p_reorder_rate
0,138,1,455775,0,6,12,1.0,10.400000,5,20,10.0,61,19,1852.0,1136.0,0.613391
1,709,1,1307498,0,0,22,6.0,7.070968,1,21,8.0,61,19,1852.0,1136.0,0.613391
2,777,1,2110387,0,1,12,7.0,12.623854,1,10,5.0,61,19,1852.0,1136.0,0.613391
3,1052,1,443227,0,1,18,8.0,10.914893,0,7,7.0,61,19,1852.0,1136.0,0.613391
4,1494,1,764206,0,3,8,23.0,6.283582,1,7,30.0,61,19,1852.0,1136.0,0.613391


### Join user features

In [30]:
features.set_index('user_id', drop = False, inplace=True)
prod_count_prior.set_index('user_id', drop = False, inplace=True)

features = features.join(prod_count_prior, rsuffix='_')
features.drop('user_id_', inplace=True, axis = 1)

features.reset_index(drop = True, inplace = True)
prod_count_prior.reset_index(drop = True, inplace = True)


features.head()

,user_id,product_id,nxt_order_id,in_next_order,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,...,aisle_id,department_id,p_orders,p_reorders,p_reorder_rate,tot_orders,tot_prods,avg_basket,avg_reorder,std_basket
0,1,196,1187899,1,4,8,14.0,20.25926,4,8,...,77,7,35791.0,27791.0,0.776480,10,59,5.9,4.1,1.523884
1,1,10258,1187899,1,4,8,14.0,20.25926,4,8,...,117,19,1946.0,1389.0,0.713772,10,59,5.9,4.1,1.523884
2,1,10326,1187899,0,4,8,14.0,20.25926,4,8,...,24,4,5526.0,3603.0,0.652009,10,59,5.9,4.1,1.523884
3,1,12427,1187899,0,4,8,14.0,20.25926,4,8,...,23,19,6476.0,4797.0,0.740735,10,59,5.9,4.1,1.523884
4,1,13032,1187899,1,4,8,14.0,20.25926,4,8,...,121,14,3751.0,2465.0,0.657158,10,59,5.9,4.1,1.523884


### Join user product features

In [31]:
features.set_index(['user_id','product_id'], drop = False, inplace=True)
user_prod_prior.set_index(['user_id','product_id'], drop = False, inplace=True)

features = features.join(user_prod_prior, rsuffix='_')
features.drop('user_id_', inplace=True, axis = 1)
features.drop('product_id_', inplace=True, axis = 1)



features.reset_index(drop = True, inplace = True)
user_prod_prior.reset_index(drop = True, inplace = True)


features.head()





,user_id,product_id,nxt_order_id,in_next_order,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,...,p_reorders,p_reorder_rate,tot_orders,tot_prods,avg_basket,avg_reorder,std_basket,up_orders,up_reorder,up_reorder_rate
0,1,196,1187899,1,4,8,14.0,20.25926,4,8,...,27791.0,0.776480,10,59,5.9,4.1,1.523884,10,9,0.900000
1,1,10258,1187899,1,4,8,14.0,20.25926,4,8,...,1389.0,0.713772,10,59,5.9,4.1,1.523884,9,8,0.888889
2,1,10326,1187899,0,4,8,14.0,20.25926,4,8,...,3603.0,0.652009,10,59,5.9,4.1,1.523884,1,0,0.000000
3,1,12427,1187899,0,4,8,14.0,20.25926,4,8,...,4797.0,0.740735,10,59,5.9,4.1,1.523884,10,9,0.900000
4,1,13032,1187899,1,4,8,14.0,20.25926,4,8,...,2465.0,0.657158,10,59,5.9,4.1,1.523884,3,2,0.666667


In [31]:
features.to_csv('./data/features_loaded.csv', index = False)

## 6. Build the model

### Vowpalwabbit - Experimental

### Write the training for vowpal wabbit model

In [ ]:
## Numpy savetxt is extremely slow
#VW_train = np.column_stack((y_train, X_train))
#print "Save"
#np.savetxt('./data/vw_train.csv', VW_train)
#print "done"

VW_train = pd.concat([Y, X],axis =1 )
print VW_train.shape
VW_train.head()





In [ ]:
print "VW_train"
VW_train.to_csv('./data/vw_train.csv', index = False)


python csv2vw.py ./data/vw_train.csv ./data/vw_train.txt 0 1
python csv2vw.py ./data/vw_test.csv ./data/vw_test.txt 0 1




### Vowpal wabbit baseline model

time vw ./data/vw_train.txt --predictions vwpred_train.out

In [ ]:
vw_pred_train = pd.read_csv('vwpred_train.out', names=['y_p'])
vw_pred_train['y_pp']= vw_pred_train['y_p'].apply(lambda x: 1.0 if x > 0.35 else 0.0)
y_p3 = vw_pred_train['y_pp'].values
print "Vowpal wabbit accuracy {0:.2f}, precision {0:.2f}, recall {0:.2f}, f1-score {0:.2f}".format(
             accuracy_score(y_train, y_p3), 
             precision_score(y_train, y_p3),
             recall_score(y_train, y_p3),
             f1_score(y_train, y_p3))

print confusion_matrix(y_train, y_p3)



## Baseline classifiers and RandomForest Model - Refer to Model_03_01 Sheet

--------------

## 7. Test Data set - Generate for final submission

In [33]:
test_list = orders[orders.eval_set == 'test']

feature = []
count = 0
for user_record in (test_list.itertuples()):
	count+=1
	if count%10000 == 0:
		print "Finished {} users".format(count)
	user_id = user_record.user_id
	order_id = user_record.order_id 
	prev_products = list(users_prior[users_prior.user_id == user_id]['prod_list'].values.tolist()[0])

	for p_p in prev_products:
		feature.append((order_id, user_id ,p_p))

test_df = pd.DataFrame(data = feature, columns =['order_id','user_id','product_id'])

print test_df.head()



Finished 10000 users
Finished 20000 users
Finished 30000 users
Finished 40000 users
Finished 50000 users
Finished 60000 users
Finished 70000 users
   order_id  user_id  product_id
0   2774568        3       17668
1   2774568        3       39190
2   2774568        3       44683
3   2774568        3       21903
4   2774568        3       14992



### Get the features  - Stats including train

In [48]:
## Order features

print "Order features"

test_df.set_index('order_id',inplace = True, drop = False)
orders.set_index('order_id', inplace = True, drop = False)

test_df = pd.merge(test_df, orders, left_on = 'order_id', right_on = 'order_id')

test_df.drop('eval_set', inplace = True, axis =1)
test_df.drop('user_id_y', inplace = True, axis =1)
test_df.drop('order_number', inplace = True, axis =1)



test_df = test_df.rename(columns={"user_id_x":"user_id"})


test_df.reset_index(drop = True, inplace= True)
train.reset_index(drop = True, inplace = True)

test_df.head()



Order features


,order_id,user_id,product_id,order_dow,order_hour_of_day,days_since_prior_order
0,2774568,3,17668,5,15,11.0
1,2774568,3,39190,5,15,11.0
2,2774568,3,44683,5,15,11.0
3,2774568,3,21903,5,15,11.0
4,2774568,3,14992,5,15,11.0


In [49]:
## Join User id order features

test_df.set_index('user_id', drop = False, inplace = True)
user_order_all.set_index('user_id', drop = False, inplace = True)

test_df = test_df.join(user_order_all,on ='user_id', rsuffix='_')
test_df.drop('user_id_', inplace = True, axis = 1)

test_df.reset_index(drop = True, inplace = True)
user_order_all.reset_index(drop = True, inplace = True)

test_df.head()

,order_id,user_id,product_id,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order
0,2774568,3,17668,5,15,11.0,11.48718
1,2774568,3,39190,5,15,11.0,11.48718
2,2774568,3,44683,5,15,11.0,11.48718
3,2774568,3,21903,5,15,11.0,11.48718
4,2774568,3,14992,5,15,11.0,11.48718


In [50]:
## User id last order feature

test_df.set_index('user_id', drop = False, inplace = True)
max_order_all.set_index('user_id', drop = False, inplace = True)

test_df = test_df.join(max_order_all, on ='user_id', rsuffix='_')
test_df.drop("user_id_", inplace = True, axis = 1)
test_df.drop("prev_order_id", inplace = True, axis = 1)

test_df.reset_index(drop = True, inplace = True)
max_order_all.reset_index(drop = True, inplace = True)

test_df.head()

,order_id,user_id,product_id,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,prev_days_since_prior_order
0,2774568,3,17668,5,15,11.0,11.48718,1,15,15.0
1,2774568,3,39190,5,15,11.0,11.48718,1,15,15.0
2,2774568,3,44683,5,15,11.0,11.48718,1,15,15.0
3,2774568,3,21903,5,15,11.0,11.48718,1,15,15.0
4,2774568,3,14992,5,15,11.0,11.48718,1,15,15.0


In [51]:
## Join Product Features
test_df.set_index('product_id', drop = False, inplace=True)
products_all.set_index('product_id', drop = False, inplace=True)


test_df = test_df.join(products_all, rsuffix='_')
test_df.drop('product_id_', inplace=True, axis = 1)

test_df.reset_index(inplace = True, drop = True)
products_all.reset_index(inplace = True, drop = True)



test_df.head()

,order_id,user_id,product_id,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,prev_days_since_prior_order,aisle_id,department_id,p_orders,p_reorders,p_reorder_rate
0,2563471,764,1,2,10,5.0,10.948718,4,17,9.0,61,19,1928.0,1185.0,0.614627
1,1930805,825,1,0,19,13.0,22.700001,1,16,20.0,61,19,1928.0,1185.0,0.614627
2,2383884,910,1,3,11,0.0,7.684210,3,10,30.0,61,19,1928.0,1185.0,0.614627
3,3099304,1379,1,3,11,13.0,17.684210,4,10,23.0,61,19,1928.0,1185.0,0.614627
4,2785463,1480,1,4,13,9.0,8.471204,2,11,13.0,61,19,1928.0,1185.0,0.614627


In [52]:
## Join user features

test_df.set_index('user_id', drop = False, inplace=True)
prod_count_all.set_index('user_id', drop = False, inplace=True)

test_df = test_df.join(prod_count_all, rsuffix='_')
test_df.drop('user_id_', inplace=True, axis = 1)

test_df.reset_index(drop = True, inplace = True)
prod_count_all.reset_index(drop = True, inplace = True)


test_df.head()

,order_id,user_id,product_id,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,prev_days_since_prior_order,aisle_id,department_id,p_orders,p_reorders,p_reorder_rate,tot_orders,tot_prods,avg_basket,avg_reorder,std_basket
0,2774568,3,248,5,15,11.0,11.48718,1,15,15.0,117,19,6590.0,2645.0,0.401366,12,88,7.333333,4.583333,2.103388
1,2774568,3,1005,5,15,11.0,11.48718,1,15,15.0,94,7,477.0,214.0,0.448637,12,88,7.333333,4.583333,2.103388
2,2774568,3,1819,5,15,11.0,11.48718,1,15,15.0,88,13,2517.0,1238.0,0.491855,12,88,7.333333,4.583333,2.103388
3,2774568,3,7503,5,15,11.0,11.48718,1,15,15.0,117,19,12922.0,7183.0,0.555874,12,88,7.333333,4.583333,2.103388
4,2774568,3,8021,5,15,11.0,11.48718,1,15,15.0,54,17,29047.0,17204.0,0.592281,12,88,7.333333,4.583333,2.103388


In [53]:
## Join user product features

test_df.set_index(['user_id','product_id'], drop = False, inplace=True)
user_prod_all.set_index(['user_id','product_id'], drop = False, inplace=True)

test_df = test_df.join(user_prod_all, rsuffix='_')
test_df.drop('user_id_', inplace=True, axis = 1)
test_df.drop('product_id_', inplace=True, axis = 1)



test_df.reset_index(drop = True, inplace = True)
user_prod_all.reset_index(drop = True, inplace = True)


test_df.head()


,order_id,user_id,product_id,order_dow,order_hour_of_day,days_since_prior_order,avg_days_since_prior_order,prev_order_dow,prev_order_hour_of_day,prev_days_since_prior_order,...,p_reorders,p_reorder_rate,tot_orders,tot_prods,avg_basket,avg_reorder,std_basket,up_orders,up_reorder,up_reorder_rate
0,2774568,3,248,5,15,11.0,11.48718,1,15,15.0,...,2645.0,0.401366,12,88,7.333333,4.583333,2.103388,1,0,0.000000
1,2774568,3,1005,5,15,11.0,11.48718,1,15,15.0,...,214.0,0.448637,12,88,7.333333,4.583333,2.103388,1,0,0.000000
2,2774568,3,1819,5,15,11.0,11.48718,1,15,15.0,...,1238.0,0.491855,12,88,7.333333,4.583333,2.103388,3,2,0.666667
3,2774568,3,7503,5,15,11.0,11.48718,1,15,15.0,...,7183.0,0.555874,12,88,7.333333,4.583333,2.103388,1,0,0.000000
4,2774568,3,8021,5,15,11.0,11.48718,1,15,15.0,...,17204.0,0.592281,12,88,7.333333,4.583333,2.103388,1,0,0.000000


In [54]:
test_df.to_csv('./data/features_submission.csv',index = False)


###  Get the features  - Only from prior

In [34]:
## Order features

print "Order features"

test_df.set_index('order_id',inplace = True, drop = False)
orders.set_index('order_id', inplace = True, drop = False)

test_df = pd.merge(test_df, orders, left_on = 'order_id', right_on = 'order_id')

test_df.drop('eval_set', inplace = True, axis =1)
test_df.drop('user_id_y', inplace = True, axis =1)
test_df.drop('order_number', inplace = True, axis =1)



test_df = test_df.rename(columns={"user_id_x":"user_id"})


test_df.reset_index(drop = True, inplace= True)
train.reset_index(drop = True, inplace = True)

features = test_df

###########test_df.head()

features.set_index('user_id', drop = False, inplace = True)
user_order_prior.set_index('user_id', drop = False, inplace = True)

features = features.join(user_order_prior,on ='user_id', rsuffix='_')
features.drop('user_id_', inplace = True, axis = 1)

features.reset_index(drop = True, inplace = True)
user_order_prior.reset_index(drop = True, inplace = True)

##########features.head()

features.set_index('user_id', drop = False, inplace = True)
max_order.set_index('user_id', drop = False, inplace = True)

features = features.join(max_order, on ='user_id', rsuffix='_')
features.drop("user_id_", inplace = True, axis = 1)
features.drop("prev_order_id", inplace = True, axis = 1)

features.reset_index(drop = True, inplace = True)
max_order.reset_index(drop = True, inplace = True)

############features.head()

features.set_index('product_id', drop = False, inplace=True)
products_prior.set_index('product_id', drop = False, inplace=True)


features = features.join(products_prior, rsuffix='_')
features.drop('product_id_', inplace=True, axis = 1)

features.reset_index(inplace = True, drop = True)
products_prior.reset_index(inplace = True, drop = True)



######features.head()

features.set_index('user_id', drop = False, inplace=True)
prod_count_prior.set_index('user_id', drop = False, inplace=True)

features = features.join(prod_count_prior, rsuffix='_')
features.drop('user_id_', inplace=True, axis = 1)

features.reset_index(drop = True, inplace = True)
prod_count_prior.reset_index(drop = True, inplace = True)


########features.head()

features.set_index(['user_id','product_id'], drop = False, inplace=True)
user_prod_prior.set_index(['user_id','product_id'], drop = False, inplace=True)

features = features.join(user_prod_prior, rsuffix='_')
features.drop('user_id_', inplace=True, axis = 1)
features.drop('product_id_', inplace=True, axis = 1)



features.reset_index(drop = True, inplace = True)
user_prod_prior.reset_index(drop = True, inplace = True)


###############features.head()

features.to_csv('./data/features_submission.csv',index = False)


Order features
